In [8]:
%load_ext aiida
%aiida

The aiida extension is already loaded. To reload it, use:
  %reload_ext aiida


Loaded AiiDA DB environment - profile name: aiida-icon-clm.

In [9]:
import io
import pendulum
import pathlib
import f90nml
import tempfile
from aiida import orm, engine, common
from aiida_c2sm import spice
from aiida import load_profile
from aiida.orm import QueryBuilder, RemoteData

load_profile("aiida-icon-clm")

Profile<uuid='6da4abb86796421db4ba41d409841450' name='aiida-icon-clm'>

In [10]:
# Why is this again defined here? Did I copy that here... -> The other file is python submission script
def get_ecraddir() -> orm.RemoteData:
    try:
        params = orm.load_node(label="spice-ecraddir")
        return params
    except common.NotExistent:
        return orm.RemoteData(
            label="spice-ecraddir",
            computer=orm.load_computer("daint-gpu"),
            # remote_path="estore/c2sm/c2sme/ICON-CLM/spice-sandbox/spice/icon-nwp-gpu/externals/ecrad/data",
            # -> Path was wrong
            remote_path="/store/c2sm/c2sme/ICON-CLM/spice-sandbox/src/icon-nwp-gpu/externals/ecrad/data",
        ).store()

In [11]:
builder = spice.gcm2icon.workflow.Gcm2Icon.get_builder()
print(builder)

{'metadata': {}, 'prep': {}, 'conv': {}, 'icon': {}, 'monitors': {}}


In [12]:
src_dir = pathlib.Path("/store/c2sm/c2sme/ICON-CLM/spice-sandbox/")

builder.boundary_data = spice.data.get_inidata()
builder.ecraddir = get_ecraddir()
builder.gcm_data = spice.data.get_gcm_data()
builder.ini_basedir = spice.data.get_inibasedir()
builder.inidata = spice.data.get_inidata()

builder.computer_options = {
    "account": "mrcloud",
    "queue_name": "normal",
    "custom_scheduler_commands": "#SBATCH -C gpu",
    "max_memory_kb": int(64e6),
    "resources": {"num_machines": 1, "num_mpiprocs_per_machine": 1},
}

# builder.prep.code = orm.load_code("spice-conv2icon-prep")  # -> Using script from SPICE fails...
builder.prep.code = orm.load_code("aiida-conv2icon-prep")  # -> Simpler script from `aiida-icon-clm`
builder.prep.computer_options = {
    "max_wallclock_seconds": 1800,
}

# builder.conv.code = orm.load_code("spice-conv2icon-conv") # -> These are the complex scripts in Michael's `/store`
# builder.conv.code = orm.load_code("spice-conv2icon-conv") # -> Using script from SPICE fails. Makes sense, as this
# should be converted to the Conv calcjobs
builder.conv.code = orm.load_code("aiida-conv2icon-conv")  # -> Simpler script from `aiida-icon-clm`
builder.conv.computer_options = {
    "max_wallclock_seconds": 1800,
}

builder.icon.code = orm.load_code("icon-gpu")
builder.icon.computer_options = {
    "max_wallclock_seconds": 3600,
    "resources": {
        "num_machines": 10,
        "num_mpiprocs_per_machine": 1,
        "num_cores_per_mpiproc": 2,
    },
    "withmpi": True,
    "mpirun_extra_params": [
        "--ntasks-per-node",
        "1",
        "--hint=nomultithread",
        "--cpus-per-task",
        "1",
    ],
}

builder.parameters = orm.JsonableData(
    spice.params.SpiceParams(
        start_date=pendulum.datetime(year=1979, month=1, day=1, tz="utc"),
        stop_date=pendulum.datetime(year=1979, month=3, day=1, tz="utc"),
        utils_bindir=src_dir / "src" / "utils" / "bin",
        cfu_bindir=src_dir / "src" / "cfu" / "bin",
        # Check if the relative paths here are fine. Full path for first file is:
        # /store/c2sm/c2sme/ICON-CLM/spice-sandbox/data/rcm/europe044/europe044_DOM01.nc
        # It should be fine, as:
        # _INIBASEDIR_PATH = "/store/c2sm/c2sme/ICON-CLM/spice-sandbox/data/rcm"
        lam_grid_relpath="europe044/europe044_DOM01.nc",
        parent_grid_relpath="europe044/europe044_DOM01.parent.nc",
        extpar_relpath="europe044/external_parameter_icon_europe044_DOM01_tiles.nc",
        ghg_file_relpath="greenhouse_gases/bc_greenhouse_rcp45_1765-2500.nc",
    )
)

In [13]:
result = engine.run(builder)

04/30/2024 04:09:45 PM <3889418> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [52|Gcm2Icon|check_inputs]: Checking inputs.
04/30/2024 04:09:45 PM <3889418> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [52|Gcm2Icon|init_iterations]: Initializing iteration variables.
04/30/2024 04:09:45 PM <3889418> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [52|Gcm2Icon|prep]: Starting Preparation run for date 1979-01-01 00:00:00.


In [ ]:
result

In [ ]:
result["remote_folder"].creator.pk

In [ ]:
from aiida.orm import load_node

gcm_data_node = load_node("gcm_data")
gcm_data_node

In [ ]:
from aiida import orm

# from aiida.engine import delete_nodes

# Initialize the QueryBuilder
qb_computer = orm.QueryBuilder()

# Add Node to the query and filter by the label
qb_computer.append(orm.Node, filters={"label": "gcm_data"}, project=["id"])

# Execute the query
# node_ids = [n[0] for n in qb.all()]

for node in qb_computer.all():
    print(node[0])